## Training the Model on Baybayin Data

Setting up Model

In [ ]:
import tensorflow as tensorflow

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import layers
from tensorflow.keras.initializers import VarianceScaling

from google.colab.patches import cv2_imshow


In [ ]:
# Defining the CNN Model
def model(num_classes, input_shape):
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(
        input_shape=input_shape,
        kernel_size=5,
        filters=8,
        strides=1,
        activation=Activation('relu'),
        kernel_initializer=VarianceScaling()
    ))
    #model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # 2nd Convolutional Layer
    model.add(Conv2D(
        kernel_size=5,
        filters=16,
        strides=1,
        activation=Activation('relu'),
        kernel_initializer=VarianceScaling()
    ))
    # Max Pooling
    model.add(MaxPooling2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # 3rd Convolutional Layer
    model.add(Conv2D(
        kernel_size=3,
        filters=24,
        strides=1,
        activation=Activation('relu'),
        kernel_initializer=VarianceScaling()
    ))
    # Max Pooling
    model.add(MaxPooling2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(
        units=128,
        activation=Activation('relu')
    ))

    # Add Dropout to prevent overfitting
    model.add(Dropout(0.2))

    # Output Layer
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))


    return model

Training the Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os

In [ ]:
!tar -xvf  './drive/MyDrive/bybyn.tar' -C './'

Streaming output truncated to the last 5000 lines.
./bybyn/ng/ng-179.jpg
./bybyn/ng/ng-18.jpg
./bybyn/ng/ng-180.jpg
./bybyn/ng/ng-181.jpg
./bybyn/ng/ng-182.jpg
./bybyn/ng/ng-19.jpg
./bybyn/ng/ng-2.jpg
./bybyn/ng/ng-20.jpg
./bybyn/ng/ng-21.jpg
./bybyn/ng/ng-22.jpg
./bybyn/ng/ng-23.jpg
./bybyn/ng/ng-24.jpg
./bybyn/ng/ng-25.jpg
./bybyn/ng/ng-26.jpg
./bybyn/ng/ng-27.jpg
./bybyn/ng/ng-28.jpg
./bybyn/ng/ng-29.jpg
./bybyn/ng/ng-3.jpg
./bybyn/ng/ng-30.jpg
./bybyn/ng/ng-31.jpg
./bybyn/ng/ng-32.jpg
./bybyn/ng/ng-33.jpg
./bybyn/ng/ng-34.jpg
./bybyn/ng/ng-35.jpg
./bybyn/ng/ng-36.jpg
./bybyn/ng/ng-37.jpg
./bybyn/ng/ng-38.jpg
./bybyn/ng/ng-39.jpg
./bybyn/ng/ng-4.jpg
./bybyn/ng/ng-40.jpg
./bybyn/ng/ng-41.jpg
./bybyn/ng/ng-42.jpg
./bybyn/ng/ng-43.jpg
./bybyn/ng/ng-44.jpg
./bybyn/ng/ng-45.jpg
./bybyn/ng/ng-46.jpg
./bybyn/ng/ng-47.jpg
./bybyn/ng/ng-48.jpg
./bybyn/ng/ng-49.jpg
./bybyn/ng/ng-5.jpg
./bybyn/ng/ng-50.jpg
./bybyn/ng/ng-51.jpg
./bybyn/ng/ng-52.jpg
./bybyn/ng/ng-53.jpg
./bybyn/ng/ng-54.jpg
./by

Settings for the Model

In [ ]:
# Define constants
DATASET_PATH = './bybyn/'
MODEL_PATH = '.'
BATCH_SIZE = 128
EPOCHS = 30
TARGET_WIDTH = 28
TARGET_HEIGHT = 28
TARGET_DEPTH = 3


In [ ]:
# Set up the data generator to flow data from disk
print("[INFO] Setting up Data Generator...")
data_gen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_generator = data_gen.flow_from_directory(
    DATASET_PATH,
    subset='training',
    target_size = (TARGET_WIDTH, TARGET_HEIGHT),
    batch_size = BATCH_SIZE
)

val_generator = data_gen.flow_from_directory(
    DATASET_PATH,
    subset='validation',
    target_size = (TARGET_WIDTH, TARGET_HEIGHT),
    batch_size = BATCH_SIZE
)

[INFO] Setting up Data Generator...
Found 7881 images belonging to 63 classes.
Found 1939 images belonging to 63 classes.


In [ ]:
# Build model
print("[INFO] Compiling model...")
bybyn = model(train_generator.num_classes, (TARGET_WIDTH, TARGET_HEIGHT, TARGET_DEPTH))

# Compile the model
bybyn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

[INFO] Compiling model...


In [ ]:
# Set the learning rate decay
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.001)



TRAIN THE MODEL

---



In [ ]:
import datetime
log_dir=".logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

training_history = bybyn.fit(
	train_generator,
	validation_data=val_generator,
	steps_per_epoch=train_generator.samples // BATCH_SIZE,
	validation_steps = val_generator.samples // BATCH_SIZE,
	epochs=EPOCHS, verbose=1, callbacks=[reduce_lr, tensorboard_callback])


Epoch 1/30
61/61 [==============================] - 6s 90ms/step - loss: 4.0889 - accuracy: 0.0235 - val_loss: 3.8336 - val_accuracy: 0.0432 - lr: 0.0010
Epoch 2/30
61/61 [==============================] - 4s 59ms/step - loss: 3.2462 - accuracy: 0.1193 - val_loss: 2.5342 - val_accuracy: 0.2448 - lr: 0.0010
Epoch 3/30
61/61 [==============================] - 4s 60ms/step - loss: 2.3741 - accuracy: 0.2358 - val_loss: 1.9744 - val_accuracy: 0.3349 - lr: 0.0010
Epoch 4/30
61/61 [==============================] - 4s 73ms/step - loss: 1.9653 - accuracy: 0.3199 - val_loss: 1.6835 - val_accuracy: 0.4078 - lr: 0.0010
Epoch 5/30
61/61 [==============================] - 4s 58ms/step - loss: 1.7530 - accuracy: 0.3719 - val_loss: 1.5251 - val_accuracy: 0.4615 - lr: 0.0010
Epoch 6/30
61/61 [==============================] - 4s 71ms/step - loss: 1.5979 - accuracy: 0.4187 - val_loss: 1.4189 - val_accuracy: 0.4839 - lr: 0.0010
Epoch 7/30
61/61 [==============================] - 3s 56ms/step - loss: 1.4

In [ ]:
# save the model to disk
print("[INFO] Serializing network...")
bybyn.save(MODEL_PATH + os.path.sep + "trained_model")

print("[INFO] Done!")

[INFO] Serializing network...
[INFO] Done!


To zip the model

In [ ]:
!tar -cvf "./drive/MyDrive/bybyn_model_trained8.tar" "./trained_model"

./trained_model/
./trained_model/assets/
./trained_model/variables/
./trained_model/variables/variables.index
./trained_model/variables/variables.data-00000-of-00001
./trained_model/keras_metadata.pb
./trained_model/fingerprint.pb
./trained_model/saved_model.pb


Zip the logs

In [ ]:
!tar -cvf "./drive/MyDrive/bybyn_model_trained8-logs.tar" "./.logs/"

./.logs/
./.logs/fit/
./.logs/fit/20230915-044656/
./.logs/fit/20230915-044656/train/
./.logs/fit/20230915-044656/train/events.out.tfevents.1694753216.cb68ed572bf8.1433.2.v2
./.logs/fit/20230915-044656/validation/
./.logs/fit/20230915-044656/validation/events.out.tfevents.1694753219.cb68ed572bf8.1433.3.v2
./.logs/fit/20230915-045006/
./.logs/fit/20230915-045006/train/
./.logs/fit/20230915-045006/train/events.out.tfevents.1694753406.cb68ed572bf8.1433.4.v2
./.logs/fit/20230915-045006/validation/
./.logs/fit/20230915-045006/validation/events.out.tfevents.1694753411.cb68ed572bf8.1433.5.v2
./.logs/fit/20230915-042317/
./.logs/fit/20230915-042317/train/
./.logs/fit/20230915-042317/train/events.out.tfevents.1694751797.cb68ed572bf8.1433.0.v2
./.logs/fit/20230915-042317/validation/
./.logs/fit/20230915-042317/validation/events.out.tfevents.1694751801.cb68ed572bf8.1433.1.v2


## Model Inference on Local Image

To unzip the model and load

In [ ]:
!tar -xvf  'bybyn_model_trained7.tar' -C './trained_model'

./trained_model/
./trained_model/keras_metadata.pb
./trained_model/fingerprint.pb
./trained_model/variables/
./trained_model/variables/variables.index
./trained_model/variables/variables.data-00000-of-00001
./trained_model/saved_model.pb
./trained_model/assets/


Setting up the trained Bybyn model

In [ ]:
import argparse
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
labels = ['a', 'b', 'ba', 'be', 'bo', 'd', 'da', 'de', 'do', 'e', 'g', 'ga', 'ge', 'go', 'h', 'ha', 'he', 'ho', 'k', 'ka', 'ke', 'ko', 'l', 'la', 'le', 'lo', 'm', 'ma', 'me', 'mo', 'n', 'na', 'ne', 'ng', 'nga', 'nge', 'ngo', 'no', 'o', 'p', 'pa', 'pe', 'po', 'r', 'ra', 're', 'ro', 's', 'sa', 'se', 'so', 't', 'ta', 'te', 'to', 'w', 'wa', 'we', 'wo', 'y', 'ya', 'ye', 'yo']

# Define constants
TARGET_WIDTH = 28
TARGET_HEIGHT = 28
MODEL_PATH = './trained_model'


In [ ]:
# Load the trained convolutional neural network
print("[INFO] Loading my model...")
model = load_model(MODEL_PATH, compile=False)

[INFO] Loading my model...


Choosing an image to classify (Must be a 28x28px JPEG Image! Black writing on white background)

In [ ]:
# Load the image
original_image = cv2.imread("./drive/MyDrive/bybyntestimgs/ko1.jpg")

In [ ]:
# Preprocessing the image
image = cv2.resize(original_image, (TARGET_WIDTH, TARGET_HEIGHT))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

cv2_imshow(original_image)

NameError: ignored

In [ ]:
print("[INFO] Classifying image...")
prob = model.predict(image)[0]
idx = np.argsort(prob)[-1]

probs = np.argsort(prob)[::-1][:5]
j = 0
for i in probs:
  print(labels[i])
  j = j+1

#print(prob)
cv2_imshow(original_image)

[INFO] Classifying image...
1/1 [==============================] - 0s 15ms/step
ke
le
ko
k
ne


In [ ]:
cv2_imshow(original_image)